In [1]:
!pip install pandas_ta backtesting yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=4d436b0caa55f771bf76c7ca5ea3ee95692db201ca28fe2d337a82512584b997
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173917 sha256=7891580c59647a5c009ebcb4554884ed47a26be50d4ad43e3be16aa031438cad
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built pandas_ta backtesting


In [2]:
from datetime import datetime
import pandas as pd
import numpy as np

import pandas_ta as ta
from backtesting import Strategy
from backtesting import Backtest
import yfinance as yf

import plotly.graph_objects as go
from plotly.subplots import make_subplots

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [37]:
data = yf.download('BTC-USD',start='2023-09-12',end='2023-10-10',interval='5m')#limit = 60 days -> create a loop to merge dataframes
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-09-12 00:00:00+00:00,25160.658203,25172.707031,25154.376953,25154.376953,25154.376953,0
2023-09-12 00:05:00+00:00,25155.212891,25173.562500,25151.646484,25173.562500,25173.562500,1625088
2023-09-12 00:10:00+00:00,25172.837891,25174.373047,25172.050781,25172.050781,25172.050781,6121472
2023-09-12 00:15:00+00:00,25178.707031,25178.707031,25156.380859,25156.380859,25156.380859,7078912
2023-09-12 00:20:00+00:00,25158.869141,25158.869141,25152.009766,25152.691406,25152.691406,0
...,...,...,...,...,...,...
2023-10-09 23:35:00+00:00,27596.779297,27596.779297,27578.451172,27580.185547,27580.185547,0
2023-10-09 23:40:00+00:00,27583.873047,27583.873047,27577.974609,27578.207031,27578.207031,516096
2023-10-09 23:45:00+00:00,27579.816406,27592.960938,27579.115234,27592.960938,27592.960938,9950208


In [67]:
df = data.copy()
df["VWAP"] = ta.vwap(df.High, df.Low, df.Close, df.Volume)
df['RSI'] =  ta.rsi(df.Close, length=7)
my_bbands = ta.bbands(df.Close, length=14, std=2.0)
df=df.join(my_bbands)
df

<ipython-input-67-96157bfbbbc1>:2: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

<ipython-input-67-96157bfbbbc1>:2: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,Open,High,Low,Close,Adj Close,Volume,VWAP,RSI,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0
Datetime,,,,,,,,,,,,,
2023-09-12 00:00:00+00:00,25160.658203,25172.707031,25154.376953,25154.376953,25154.376953,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-12 00:05:00+00:00,25155.212891,25173.562500,25151.646484,25173.562500,25173.562500,1625088,25166.257161,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-12 00:10:00+00:00,25172.837891,25174.373047,25172.050781,25172.050781,25172.050781,6121472,25171.447084,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-12 00:15:00+00:00,25178.707031,25178.707031,25156.380859,25156.380859,25156.380859,7078912,25167.806673,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-12 00:20:00+00:00,25158.869141,25158.869141,25152.009766,25152.691406,25152.691406,0,25167.806673,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-09 23:35:00+00:00,27596.779297,27596.779297,27578.451172,27580.185547,27580.185547,0,27575.440357,37.444002,27572.996303,27600.931501,27628.866699,0.202422,0.128677
2023-10-09 23:40:00+00:00,27583.873047,27583.873047,27577.974609,27578.207031,27578.207031,516096,27575.440780,36.376944,27576.718586,27596.447684,27616.176782,0.142983,0.037722
2023-10-09 23:45:00+00:00,27579.816406,27592.960938,27579.115234,27592.960938,27592.960938,9950208,27575.463700,49.016919,27576.708944,27595.348912,27613.988879,0.135095,0.435945


In [75]:

backcandles = 21

def getVWAPSignal(df):
  VWAPsignal = [0]*len(df)
  for row in range(backcandles, len(df)):
      upt = 1
      dnt = 1
      for i in range(row-backcandles, row+1):
          if max(df.Open[i], df.Close[i])>=df.VWAP[i]:
              dnt=0
          if min(df.Open[i], df.Close[i])<=df.VWAP[i]:
              upt=0
      if upt==1 and dnt==1:
          VWAPsignal[row]=3
      elif upt==1:
          VWAPsignal[row]=2
      elif dnt==1:
          VWAPsignal[row]=1

  return VWAPsignal
df['VWAPSignal'] = getVWAPSignal(df)
df['VWAPSignal'].tail()

2023-10-09 23:35:00+00:00    2
2023-10-09 23:40:00+00:00    2
2023-10-09 23:45:00+00:00    2
2023-10-09 23:50:00+00:00    2
2023-10-09 23:55:00+00:00    2
Name: VWAPSignal, dtype: int64

In [76]:
def TotalSignal(l):
    if (df.VWAPSignal[l]==2
        and df.Close[l]<=df['BBL_14_2.0'][l]
        and df.RSI[l]<45):
            return 2
    if (df.VWAPSignal[l]==1
        and df.Close[l]>=df['BBU_14_2.0'][l]
        and df.RSI[l]>55):
            return 1
    return 0

TotSignal = [0]*len(df)
for row in range(backcandles, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

In [21]:
df[df.TotalSignal!=0].count()

Open           135
High           135
Low            135
Close          135
Adj Close      135
Volume         135
VWAP           133
RSI            135
BBL_14_2.0     135
BBM_14_2.0     135
BBU_14_2.0     135
BBB_14_2.0     135
BBP_14_2.0     135
VWAPSignal     135
TotalSignal    135
dtype: int64

In [77]:
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [78]:
# st=10400
dfpl = df
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP,
                           line=dict(color='blue', width=1),
                           name="VWAP"),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'],
                           line=dict(color='green', width=1),
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'],
                           line=dict(color='green', width=1),
                           name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=6, color="MediumPurple"),
                name="Signal")
fig.update_layout(height=800)
fig.show()

In [86]:
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)
def SIGNAL():
    return dfpl.TotalSignal

In [87]:
class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio = 1.5

        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=80:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=20:
                self.trades[-1].close()

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/10, commission=0.00)
stat = bt.run()
stat

Start                     2023-09-12 00:00...
End                       2023-10-09 23:55...
Duration                     27 days 23:55:00
Exposure Time [%]                    7.935917
Equity Final [$]                114174.247187
Equity Peak [$]                 121489.834232
Return [%]                          14.174247
Buy & Hold Return [%]                 9.66082
Return (Ann.) [%]                   74.586576
Volatility (Ann.) [%]               82.358555
Sharpe Ratio                         0.905632
Sortino Ratio                        2.553953
Calmar Ratio                         5.197083
Max. Drawdown [%]                  -14.351622
Avg. Drawdown [%]                   -4.208383
Max. Drawdown Duration       26 days 06:25:00
Avg. Drawdown Duration        5 days 09:24:00
# Trades                                  131
Win Rate [%]                        44.274809
Best Trade [%]                       0.628265
Worst Trade [%]                     -0.412671
Avg. Trade [%]                    

In [81]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/usr/l

GridPlot(id='p3047', ...)